### 1. MTBS Binary Export
Draw boxes on map one at a time and export them to google drive

In [1]:
import ee
import geemap

In [8]:
ee.Authenticate()

Enter verification code: 4/1AY0e-g5RqXIkfRjKmcBJSm4mg5D1K00UuF05q81HSwu-cLxOwHPhFegL7gI

Successfully saved authorization token.


In [9]:
ee.Initialize()

In [4]:
# Grab MTBS data I've uploaded as an asset
d_0 = ee.Image('users/morganrsteckler/gee/mtbs_2015_0')
d_1 = ee.Image('users/morganrsteckler/gee/mtbs_2015_1')
d_2 = ee.Image('users/morganrsteckler/gee/mtbs_2015_2')

In [5]:
# Create map and draw polygon
Map = geemap.Map()
Map.addLayer(d_0, {}, "mtbs_2015_0")
Map.center_object(d_0, zoom=5)
Map

EEException: Image.load: Image asset 'users/morganrsteckler/gee/mtbs_2015_0' not found.

In [44]:
# Access the polygon
Map.draw_features

[]

In [46]:
# Create feature collection and clip fires to collection
roi = ee.FeatureCollection(Map.draw_features)
fire = dataset_dqr.clip(roi)

In [47]:
# Remove mask so Nones are 0
fire = fire.unmask(0)

In [48]:
# Calculate the area of the ROI
def addArea(feat):
    area = feat.geometry().area(10)
    return feat.set('area', area)

In [49]:
# Add area to properties and show it's there
added = addArea(roi)
info = added.getInfo()
print(info)

EEException: Collection.geometry, argument 'collection': Invalid type.
Expected type: FeatureCollection.
Actual type: Image<[b1]>.

In [54]:
# Function to turn vector into raster based on area
def makeRast(fc):
    img = (fc
           .reduceToImage(properties=['area'],
                      reducer=ee.Reducer.first())
      )
    return img

In [55]:
# Turn vector into raster
img = makeRast(added)

In [56]:
# Create dummy raster with all zeros
zero = ee.Image(0)
print(zero.getInfo())

{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 0}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [57]:
# Add dummy bands to raster
img = img.addBands(zero)
print(img.getInfo())

{'type': 'Image', 'bands': [{'id': 'first', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 0}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [58]:
# Sum zeros and fire
square = img.select('constant')
fire = fire.select('b1')
summed = (square.add(fire)).clip(roi)

In [59]:
# Set fires to 1 and visualize
summed = summed.expression("B1 == 0 ? 0 : 1", 
                           {'B1': summed.select('constant')}
                          ).clip(roi)
Map.addLayer(summed, {}, 'summed')

In [60]:
# Select just geometry from FC
geom = roi.first().geometry()
geom = ee.Geometry(geom)
print(geom.getInfo())

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-157.213708, 65.473444], [-157.213708, 67.618962], [-151.841175, 67.618962], [-151.841175, 65.473444], [-157.213708, 65.473444]]]}


In [61]:
# Export to drive as tif
geemap.ee_export_image_to_drive(summed, 
                                description='mtbs_2015_2', 
                                folder='MTBS', 
                                region=geom, 
                                scale=30)

Exporting mtbs_2015_2 ...


### 2. MODIS NDVI and LST export

In [6]:
# Create map and draw polygon
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [10]:
dataset_yia = ee.Image('users/morganrsteckler/mtbs_2015_2')
geom_yia = dataset_yia.geometry()
geom_2 = ee.Geometry(geom_yia)
Map.addLayer(dataset_yia, {}, "mtbs_2015_2")

In [13]:
dataset_fwb = ee.Image('users/morganrsteckler/mtbs_2015_1')
geom_fwb = dataset_fwb.geometry()
geom_1 = ee.Geometry(geom_fwb)
Map.addLayer(dataset_fwb, {}, "mtbs_2015_1")

In [14]:
dataset_zad = ee.Image('users/morganrsteckler/mtbs_2015_0')
geom_zad = dataset_zad.geometry()
geom_0 = ee.Geometry(geom_zad)
Map.addLayer(dataset_zad, {}, "mtbs_2015_0")

In [11]:
# Import MODIS data for LST and calculating NDVI
def import_sr(geom):
    """Function to import 8-day MODIS SR for varying geometries."""
    sr = (ee.ImageCollection("MODIS/006/MOD09A1")
          .filterDate('2015-05-01', '2015-08-31')
          .filterBounds(geom)
          .select(['sur_refl_b02', 'sur_refl_b01'])
         )
    return sr

In [12]:
def import_lst(geom):
    """Function to import 8-day MODIS LST for varying geometries."""
    lst = (ee.ImageCollection("MODIS/006/MOD11A2")
           .filterDate('2015-05-01', '2015-08-31')
           .filterBounds(geom)
           .select(['LST_Day_1km'])
          )
    return lst

In [17]:
# Calculate NDVI
def ndvi_band(img):
    """Function to calculate ndre for an image."""
    ndvi = img.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')
    return img.addBands(ndvi)

# Define NDRE palette
ndvi_palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', 
           '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

In [18]:
# Import surface reflectance for geometries
sr0 = import_sr(geom_0)
sr1 = import_sr(geom_1)
sr2 = import_sr(geom_2)

In [19]:
# Calculate NDVI
ndvi_0 = (sr0.map(ndvi_band)).select('NDVI')
ndvi_1 = (sr1.map(ndvi_band)).select('NDVI')
ndvi_2 = (sr2.map(ndvi_band)).select('NDVI')

In [20]:
# Clip to geometry and visualize
ndvi_0 = ndvi_0.map(lambda img: img.clip(geom_0))
ndvi_1 = ndvi_1.map(lambda img: img.clip(geom_1))
ndvi_2 = ndvi_2.map(lambda img: img.clip(geom_2))

Map.addLayer(ndvi_2.max(), {'min': -1, 'max': 1, 'palette': ndvi_palette}, 'ndvi2')

In [21]:
# Import LST for geometries
lst1 = import_lst(geom_0)
lst2 = import_lst(geom_1)
lst3 = import_lst(geom_2)

In [22]:
# Clip to geometry and visualize
lst_0 = lst1.map(lambda img: img.clip(geom_0))
lst_1 = lst2.map(lambda img: img.clip(geom_1))
lst_2 = lst3.map(lambda img: img.clip(geom_2))

palette = [ '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
           '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
           '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
           'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
           'ff0000', 'de0101', 'c21301', 'a71001', '911003'
          ]

Map.addLayer(lst_2.median(), {'min': 14000, 'max': 16000, 'palette': palette}, 'lst2')

In [23]:
print(ee.Projection('EPSG:3857').getInfo())
print(ee.Projection('EPSG:4326').getInfo())

{'type': 'Projection', 'crs': 'EPSG:3857', 'transform': [1, 0, 0, 0, 1, 0]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [24]:
def reproj(img):
    proj = ee.Projection('EPSG:3338')
    return (img.reproject(crs=proj, scale=500))

ndvi_0 = ndvi_0.map(reproj)
ndvi_1 = ndvi_1.map(reproj)
ndvi_2 = ndvi_2.map(reproj)
lst_0 = lst_0.map(reproj)
lst_1 = lst_1.map(reproj)
lst_2 = lst_2.map(reproj)

In [25]:
ndvi_2 = ndvi_2.map(lambda img: img.clip(geom_2))
lst_2 = lst_2.map(lambda img: img.clip(geom_2))

In [26]:
# Create tif names
lst = [ndvi_2]
for i in range(0,len(lst)):
    OUT = lst[i]
    OUT_NAME = "ndvi{}".format(i)

    size = (OUT.size()).getInfo()

    descs = []
    for j in range(0, size):
        desc = OUT_NAME + "_{}_3338".format(j)
        descs.append(desc)
    print(descs)

    # Export to drive as tifs
    geemap.ee_export_image_collection_to_drive(OUT,
                                               descriptions=descs,
                                               folder='MTBS', 
                                               scale=500, 
                                               crs='EPSG:3338')

['ndvi0_0_3338', 'ndvi0_1_3338', 'ndvi0_2_3338', 'ndvi0_3_3338', 'ndvi0_4_3338', 'ndvi0_5_3338', 'ndvi0_6_3338', 'ndvi0_7_3338', 'ndvi0_8_3338', 'ndvi0_9_3338', 'ndvi0_10_3338', 'ndvi0_11_3338', 'ndvi0_12_3338', 'ndvi0_13_3338', 'ndvi0_14_3338', 'ndvi0_15_3338']
Total number of images: 16

Exporting ndvi0_0_3338 ...
Exporting ndvi0_1_3338 ...
Exporting ndvi0_2_3338 ...
Exporting ndvi0_3_3338 ...
Exporting ndvi0_4_3338 ...
Exporting ndvi0_5_3338 ...
Exporting ndvi0_6_3338 ...
Exporting ndvi0_7_3338 ...
Exporting ndvi0_8_3338 ...
Exporting ndvi0_9_3338 ...
Exporting ndvi0_10_3338 ...
Exporting ndvi0_11_3338 ...
Exporting ndvi0_12_3338 ...
Exporting ndvi0_13_3338 ...
Exporting ndvi0_14_3338 ...
Exporting ndvi0_15_3338 ...


In [27]:
# Create tif names
lst = [lst_2]
for i in range(0, len(lst)):
    OUT = lst[i]
    OUT_NAME = "lst{}".format(i)

    size = (OUT.size()).getInfo()

    descs = []
    for j in range(0, size):
        desc = OUT_NAME + "_{}_3338".format(j)
        descs.append(desc)
    print(descs)

    # Export to drive as tifs
    geemap.ee_export_image_collection_to_drive(OUT,
                                               descriptions=descs,
                                               folder='MTBS', 
                                               scale=1000,
                                               crs='EPSG:3338')

['lst0_0_3338', 'lst0_1_3338', 'lst0_2_3338', 'lst0_3_3338', 'lst0_4_3338', 'lst0_5_3338', 'lst0_6_3338', 'lst0_7_3338', 'lst0_8_3338', 'lst0_9_3338', 'lst0_10_3338', 'lst0_11_3338', 'lst0_12_3338', 'lst0_13_3338', 'lst0_14_3338', 'lst0_15_3338']
Total number of images: 16

Exporting lst0_0_3338 ...
Exporting lst0_1_3338 ...
Exporting lst0_2_3338 ...
Exporting lst0_3_3338 ...
Exporting lst0_4_3338 ...
Exporting lst0_5_3338 ...
Exporting lst0_6_3338 ...
Exporting lst0_7_3338 ...
Exporting lst0_8_3338 ...
Exporting lst0_9_3338 ...
Exporting lst0_10_3338 ...
Exporting lst0_11_3338 ...
Exporting lst0_12_3338 ...
Exporting lst0_13_3338 ...
Exporting lst0_14_3338 ...
Exporting lst0_15_3338 ...
